In [183]:
import requests
from bs4 import BeautifulSoup

url = "https://finance.naver.com/marketindex/?tabSel=gold#tab_section"

response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

global_gold = 'gold_inter' # per toz: 1트로이온스(Troy ounce) = 31.1034768g
domestic_gold = 'gold_domestic'
krwusd = 'usd'

def get_current_prev_price(head_str):
    current_price = float(soup.find_all('a', class_=f'head {head_str}')[0].select('span.value')[0].text.replace(',',''))
    price_change = float(soup.find_all('a', class_=f'head {head_str}')[0].select('span.change')[0].text.replace(',',''))
    direction = soup.find_all('a', class_=f'head {head_str}')[0].select('span.blind')[-1].text
    direction_int = 1 if '상승' == direction else -1
    prev_price = current_price + price_change * direction_int

    prices_ser = pd.Series({
        'current_price': current_price,
        'prev_price': prev_price
    })
    return prices_ser

In [252]:
global_gld_prcs = get_current_prev_price(global_gold) # 국제 금 시세
domestic_gld_prcs = get_current_prev_price(domestic_gold) # 국내 금 시세
krwusd_prcs = get_current_prev_price(krwusd) # 원화 환율

global_gld_krw = global_gld_prcs * krwusd_prcs / 31.1034768
summary_df = pd.concat([domestic_gld_prcs.rename('domestic_gld_prcs'), global_gld_krw.rename('global_gld_krw')], axis=1)
summary_df['premium_krw'] = summary_df.domestic_gld_prcs - summary_df.global_gld_krw
summary_df['premium_pct'] = (summary_df.domestic_gld_prcs / summary_df.global_gld_krw - 1) * 100

# 1g당 금시세 차이
summary_df.T.style.format("{:,.2f}")

,current_price,prev_price
domestic_gld_prcs,"145,619.68","143,989.59"
global_gld_krw,"148,717.10","152,320.49"
premium_krw,"-3,097.42","-8,330.90"
premium_pct,-2.08,-5.47
